In [ ]:
import scqubits as scq
import numpy as np
import plotly.graph_objects as go
import pandas as pd

In [ ]:
qub_name = "S001"

loadpath = f"../result/{qub_name}/params.csv"
df = pd.read_csv(loadpath)
cflx, period = df.loc[0, ["Half flux", "Period"]]
EJ, EC, EL = df.loc[0, ["EJ", "EC", "EL"]]
df.head()

In [ ]:
r_f = 7.48675
g = 0.034


In [ ]:
flxs = np.linspace(0, 0.51, 1001)

fluxonium = scq.Fluxonium(EJ, EC, EL, flux=0.5, cutoff=30, truncated_dim=5)
resonator = scq.Oscillator(r_f, truncated_dim=5)
hilbertspace = scq.HilbertSpace([fluxonium, resonator])
hilbertspace.add_interaction(
    g=g, op1=fluxonium.n_operator, op2=resonator.creation_operator, add_hc=True
)


def update_hilbertspace(flux):
    fluxonium.flux = flux


param_name = r"$Φ_{ext}/Φ_0$"
sweep = scq.ParameterSweep(
    hilbertspace,
    {param_name: flxs},
    update_hilbertspace=update_hilbertspace,
    evals_count=20,
    subsys_update_info={param_name: [fluxonium]},
)
dispersives = sweep["chi"]["subsys1":0, "subsys2":1][:, 1].toarray() * 1e3

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=flxs,
        y=dispersives,
        mode="lines",
        line=dict(color="blue"),
        name=r"$\chi_{01}$",
    )
)
fig.add_hline(y=0, line_dash="dash", line_color="black")

fig.update_layout(
    xaxis_title=param_name,
    yaxis_title=r"$\chi_{01} (MHz)$",
    title=r"$\chi_{01} vs. \Phi_{ext}/\Phi_0$",
    title_x=0.5,
)
fig.write_html(f"../result/{qub_name}/web/dispersive_01.html", include_plotlyjs="cdn")
fig.show()